In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import TimeSeriesSplit

def prepare_xy_time_series(
    df: pd.DataFrame,
    date_col: str,
    target_col: str,
    horizon: int = 1,          # ทำนายล่วงหน้า k step
    drop_cols: list[str] | None = None
):
    """
    สร้าง y = target ในอนาคต (t + horizon) และ X = features ณ เวลา t
    """
    df = df.sort_values(date_col).reset_index(drop=True).copy()

    # สร้าง label อนาคต
    df["__y__"] = df[target_col].shift(-horizon)

    # drop แถวท้ายที่ไม่มี y
    df = df.dropna(subset=["__y__"]).reset_index(drop=True)

    # สร้าง X
    drop_cols = (drop_cols or []) + [target_col]  # ปกติไม่ให้ใช้ target ปัจจุบันเป็น feature
    X = df.drop(columns=[c for c in drop_cols if c in df.columns] + ["__y__"])
    y = df["__y__"]

    # เก็บวันที่เผื่อ debug
    t = df[date_col].copy()

    # แนะนำ: ตรวจคอลัมน์ที่ไม่ใช่ตัวเลข
    # X = X.select_dtypes(include=[np.number])

    return X, y, t


c:\Users\ssupa\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\_param_validation.py:14: UserWarning: A NumPy version >=1.22.4 and <2.3.0 is required for this version of SciPy (detected version 2.3.0)
  from scipy.sparse import csr_matrix, issparse


In [2]:
def model_based_importance(model, feature_names):
    if not hasattr(model, "feature_importances_"):
        raise ValueError("Model นี้ไม่มี feature_importances_")
    imp = pd.Series(model.feature_importances_, index=feature_names).sort_values(ascending=False)
    return imp


In [3]:
from sklearn.inspection import permutation_importance
from sklearn.metrics import mean_absolute_error, mean_squared_error

def _rmse(y_true, y_pred):
    return float(np.sqrt(mean_squared_error(y_true, y_pred)))

def permutation_importance_timeseries(
    model_factory,                 # ฟังก์ชันที่คืน model ใหม่ทุก fold
    X: pd.DataFrame,
    y: pd.Series,
    n_splits: int = 5,
    n_repeats: int = 10,
    scoring: str = "neg_root_mean_squared_error",  # regression: "neg_mean_absolute_error" ก็ได้
    random_state: int = 42,
):
    tscv = TimeSeriesSplit(n_splits=n_splits)
    all_imps = []

    for fold, (tr, te) in enumerate(tscv.split(X), start=1):
        Xtr, Xte = X.iloc[tr], X.iloc[te]
        ytr, yte = y.iloc[tr], y.iloc[te]

        model = model_factory()
        model.fit(Xtr, ytr)

        r = permutation_importance(
            model, Xte, yte,
            n_repeats=n_repeats,
            random_state=random_state,
            scoring=scoring,
        )
        fold_imp = pd.Series(r.importances_mean, index=X.columns, name=f"fold{fold}")
        all_imps.append(fold_imp)

    imp_df = pd.concat(all_imps, axis=1)
    summary = pd.DataFrame({
        "importance_mean": imp_df.mean(axis=1),
        "importance_std": imp_df.std(axis=1),
    }).sort_values("importance_mean", ascending=False)

    return summary, imp_df


In [4]:
from sklearn.base import clone

def block_permutation_importance(
    model,
    X_test: pd.DataFrame,
    y_test: pd.Series,
    metric_fn,                  # ฟังก์ชัน metric: ยิ่งน้อยยิ่งดี เช่น RMSE/MAE
    block_size: int = 24,       # ตัวอย่าง: รายชั่วโมง 24 = 1 วัน
    n_repeats: int = 20,
    random_state: int = 42
):
    rng = np.random.default_rng(random_state)

    # baseline
    base_pred = model.predict(X_test)
    base_score = metric_fn(y_test, base_pred)

    n = len(X_test)
    n_blocks = int(np.ceil(n / block_size))
    blocks = [np.arange(i*block_size, min((i+1)*block_size, n)) for i in range(n_blocks)]

    importances = {}
    for col in X_test.columns:
        deltas = []
        for _ in range(n_repeats):
            perm_blocks = blocks.copy()
            rng.shuffle(perm_blocks)

            Xp = X_test.copy()
            # เอาค่าจากบล็อกเดิม ไปวางตามลำดับบล็อกใหม่
            src_idx = np.concatenate(blocks)
            dst_idx = np.concatenate(perm_blocks)

            Xp.iloc[dst_idx, Xp.columns.get_loc(col)] = X_test.iloc[src_idx, X_test.columns.get_loc(col)].values

            pred = model.predict(Xp)
            score = metric_fn(y_test, pred)
            deltas.append(score - base_score)  # เพิ่มขึ้นเท่าไร = สำคัญเท่านั้น

        importances[col] = (float(np.mean(deltas)), float(np.std(deltas)))

    out = pd.DataFrame(importances, index=["importance_mean", "importance_std"]).T
    out = out.sort_values("importance_mean", ascending=False)
    return out, base_score


In [5]:
def drop_column_importance_timeseries(
    model_factory,
    X: pd.DataFrame,
    y: pd.Series,
    metric_fn,               # ยิ่งน้อยยิ่งดี (RMSE/MAE)
    n_splits: int = 5,
):
    tscv = TimeSeriesSplit(n_splits=n_splits)

    # ใช้ fold สุดท้ายเป็น "อนาคต" ที่ชัดสุด
    folds = list(tscv.split(X))
    tr, te = folds[-1]
    Xtr, Xte = X.iloc[tr], X.iloc[te]
    ytr, yte = y.iloc[tr], y.iloc[te]

    # baseline
    base_model = model_factory()
    base_model.fit(Xtr, ytr)
    base_score = metric_fn(yte, base_model.predict(Xte))

    results = []
    for col in X.columns:
        Xm_tr = Xtr.drop(columns=[col])
        Xm_te = Xte.drop(columns=[col])

        m = model_factory()
        m.fit(Xm_tr, ytr)
        score = metric_fn(yte, m.predict(Xm_te))

        results.append((col, score - base_score))  # คะแนนแย่ลงเท่าไร

    out = pd.DataFrame(results, columns=["feature", "importance"])
    out = out.sort_values("importance", ascending=False).reset_index(drop=True)
    return out, base_score


In [6]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

DATE_COL = "Date"         # แก้ให้ตรงไฟล์คุณ
TARGET_COL = "PM2.5"      # แก้ให้ตรงไฟล์คุณ

df = pd.read_csv(r"C:\Users\ssupa\Code\Pm2.5_forcast\data\44T_cleaned_features.csv")          # หรือ path ของคุณ
df[DATE_COL] = pd.to_datetime(df[DATE_COL])

X, y, t = prepare_xy_time_series(
    df, date_col=DATE_COL, target_col=TARGET_COL, horizon=1,
    drop_cols=[DATE_COL]  # ปกติไม่ให้ date เป็นฟีเจอร์ตรง ๆ (แต่คุณมี sin/cos แล้ว)
)

def model_factory():
    return RandomForestRegressor(
        n_estimators=500,
        random_state=42,
        n_jobs=-1
    )

# 1) Permutation importance (cross-validated)
pi_summary, pi_by_fold = permutation_importance_timeseries(
    model_factory, X, y,
    n_splits=5,
    n_repeats=10,
    scoring="neg_mean_absolute_error"
)
print(pi_summary.head(20))

# 2) Block permutation บน fold สุดท้าย (สไตล์ time series)
from sklearn.model_selection import TimeSeriesSplit
tscv = TimeSeriesSplit(n_splits=5)
tr, te = list(tscv.split(X))[-1]
m = model_factory()
m.fit(X.iloc[tr], y.iloc[tr])

block_imp, base = block_permutation_importance(
    m, X.iloc[te], y.iloc[te],
    metric_fn=lambda yt, yp: mean_absolute_error(yt, yp),
    block_size=24,    # ปรับตามความถี่ข้อมูล
    n_repeats=20
)
print("baseline MAE =", base)
block_imp
# print(block_imp.head(100))


                  importance_mean  importance_std
PM2.5_lag1               0.284863        0.167426
PM2.5_rollmean7          0.268975        0.142930
PM2.5_rollmean14         0.261068        0.554325
PM2.5_rollmean3          0.217118        0.112811
dayofyear_cos            0.079973        0.104690
Humi.                    0.078240        0.124104
PM2.5_lag14              0.035464        0.066248
Pres.                    0.033497        0.051571
Temp.                    0.028698        0.049032
PM2.5_lag3               0.028078        0.021437
PM2.5_lag2               0.024239        0.022766
Wind Dir_sin             0.017846        0.038397
Wind Dir_cos             0.015246        0.014175
Prec.                    0.014960        0.022366
Wind Speed               0.013645        0.041516
PM2.5_lag30              0.013445        0.025870
PM2.5_lag7               0.009098        0.027614
heatidx                  0.007690        0.044980
Prec._lag1               0.007591        0.011535


,importance_mean,importance_std
PM2.5_rollmean14,1.133364,0.537164
PM2.5_rollmean7,0.465438,0.198673
PM2.5_lag1,0.458942,0.166660
PM2.5_rollmean3,0.353995,0.095147
PM2.5_lag14,0.134347,0.076884
dayofyear_cos,0.075480,0.047230
PM2.5_lag30,0.059036,0.038390
PM2.5_lag3,0.045308,0.018017
PM2.5_lag2,0.044297,0.016849
PM2.5_rollstd7,0.039975,0.035426
